In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.datasets import cifar100
from keras import applications
from __future__ import print_function
from keras import optimizers
from keras.models import Sequential,Input,Model
from keras.models import load_model #save and load models
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.preprocessing import LabelEncoder
import keras.backend as k

In [2]:
df = pd.read_csv('uber_census_extra_features.csv', encoding = "ISO-8859-1", engine='python')
data = df
print(data.shape)
print(data.dtypes)
df1 = data

(9500, 14)
hour                            int64
lat                           float64
long                          float64
base                           object
MedianIncomeByPlaceofBirth    float64
MedianIncome                  float64
TotalPopulation               float64
AvgPopulation                 float64
Population5M                  float64
Population45M                 float64
Population4M                  float64
Population35M                 float64
Population3M                  float64
UserGroup                       int64
dtype: object


In [3]:
df1.head()

,hour,lat,long,base,MedianIncomeByPlaceofBirth,MedianIncome,TotalPopulation,AvgPopulation,Population5M,Population45M,Population4M,Population35M,Population3M,UserGroup
0,56,40.7564,-73.9762,B02512,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0
1,27,40.7552,-73.9775,B02617,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0
2,6,40.7561,-73.9788,B02617,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0
3,28,40.7538,-73.9774,B02512,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0
4,14,40.6710,-73.9110,B02617,9882.0,10693.0,3857.0,4017.0,2786339.0,2294032.0,1923713.0,1604816.0,1315883.0,0


In [4]:
names_cloud = data.columns.tolist()
print(names_cloud)
X = np.array(data[names_cloud])
print(X.shape)

['hour', 'lat', 'long', 'base', 'MedianIncomeByPlaceofBirth', 'MedianIncome', 'TotalPopulation', 'AvgPopulation', 'Population5M', 'Population45M', 'Population4M', 'Population35M', 'Population3M', 'UserGroup']
(9500, 14)


In [5]:
# fig = plt.figure(figsize=(20,20))
# cols = 5
# rows = (float(data.shape[1]) / cols)
# for i, column in enumerate(data.columns):
#     a = fig.add_subplot(rows, cols, i + 1)
#     a.set_title(column)
#     if data.dtypes[column] == np.object:
#         data[column].value_counts().plot(kind="bar", axes=a)
#     else:
#         data[column].hist(axes=a)
#         plt.xticks(rotation="vertical")
# plt.subplots_adjust(hspace=0.7, wspace=0.2)

In [6]:
y = (data['UserGroup']).values
print(pd.value_counts(pd.Series(y)))
data.drop('UserGroup',axis=1, inplace =True,)

1    4788
0    4712
dtype: int64


In [7]:
categorical_features = data.select_dtypes(include=['object']).columns
print(categorical_features)
ohc_category = ['base']
df_ohc = pd.get_dummies(data, columns = ohc_category)
print(df_ohc.shape)
df_ohc.head()

Index(['base'], dtype='object')
(9500, 15)


,hour,lat,long,MedianIncomeByPlaceofBirth,MedianIncome,TotalPopulation,AvgPopulation,Population5M,Population45M,Population4M,Population35M,Population3M,base_B02512,base_B02598,base_B02617
0,56,40.7564,-73.9762,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,1,0,0
1,27,40.7552,-73.9775,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0,0,1
2,6,40.7561,-73.9788,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0,0,1
3,28,40.7538,-73.9774,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,1,0,0
4,14,40.6710,-73.9110,9882.0,10693.0,3857.0,4017.0,2786339.0,2294032.0,1923713.0,1604816.0,1315883.0,0,0,1


In [8]:
names_x = df_ohc.columns.tolist()
print("Target Variable: UserGroup")
print("Predictors: "+str(names_x))
x = np.array(df_ohc[names_x])
print("Number of data samples : {0:d}".format(x.shape[0]))
print("Number of Predictor Features : {0:d}".format(x.shape[1]))
#df_ohc["Age"] = df_ohc["Age"].astype(str).astype(float)
# from google.colab import files
# df_ohc.to_csv('df.csv')
# files.download('df.csv')
df_ohc[names_x]

Target Variable: UserGroup
Predictors: ['hour', 'lat', 'long', 'MedianIncomeByPlaceofBirth', 'MedianIncome', 'TotalPopulation', 'AvgPopulation', 'Population5M', 'Population45M', 'Population4M', 'Population35M', 'Population3M', 'base_B02512', 'base_B02598', 'base_B02617']
Number of data samples : 9500
Number of Predictor Features : 15


,hour,lat,long,MedianIncomeByPlaceofBirth,MedianIncome,TotalPopulation,AvgPopulation,Population5M,Population45M,Population4M,Population35M,Population3M,base_B02512,base_B02598,base_B02617
0,56,40.7564,-73.9762,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,1,0,0
1,27,40.7552,-73.9775,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0,0,1
2,6,40.7561,-73.9788,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,0,0,1
3,28,40.7538,-73.9774,112875.0,-666666666.0,56.0,44.0,2961953.0,2468816.0,2150250.0,1825733.0,1454349.0,1,0,0
4,14,40.6710,-73.9110,9882.0,10693.0,3857.0,4017.0,2786339.0,2294032.0,1923713.0,1604816.0,1315883.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,15,40.9073,-73.8125,93828.0,248438.0,1653.0,1593.0,971237.0,770740.0,628319.0,470894.0,403638.0,0,0,1
9496,14,40.7168,-74.0148,129028.0,250001.0,5644.0,8891.0,2247012.0,1810662.0,1545796.0,1235742.0,949452.0,0,0,1
9497,3,40.7146,-74.0071,141635.0,250001.0,6863.0,6488.0,2438407.0,1973620.0,1614087.0,1338260.0,1002372.0,0,1,0
9498,26,40.7140,-74.0148,129028.0,250001.0,5644.0,8891.0,2247012.0,1810662.0,1545796.0,1235742.0,949452.0,0,1,0


In [9]:
df_ohc.dtypes

hour                            int64
lat                           float64
long                          float64
MedianIncomeByPlaceofBirth    float64
MedianIncome                  float64
TotalPopulation               float64
AvgPopulation                 float64
Population5M                  float64
Population45M                 float64
Population4M                  float64
Population35M                 float64
Population3M                  float64
base_B02512                     uint8
base_B02598                     uint8
base_B02617                     uint8
dtype: object

In [10]:
x = x.astype('float32')
x = x/255
batch_size = 64 #upto us
epochs = 200
lrate = 0.001
decay = 1e-7 
data_size = 2250
ns = 10 #number of shadow models for one data_size
nh = 8 #number of hidden layers
nout = 1
seed = 9
np.random.seed(seed)
sh = np.arange(x.shape[0])
np.random.shuffle(sh)
target_rep = np.zeros((1,x.shape[0]))
target_rep[0,:] = sh
print(sh)

[5866 6473  759 ... 6782 4444 8574]


In [11]:
from tensorflow.keras.optimizers import Adam
k.clear_session()
xtr_target = x[sh[:data_size]]
ytr_target = y[sh[:data_size]]
xts_target = x[sh[data_size:data_size*2]]
yts_target = y[sh[data_size:2*data_size]]
shadow_rep = np.zeros((20,x.shape[0]-2*data_size))
sh1 = sh[2*data_size:]
xtr_att = np.zeros((2*data_size*ns,1))
ytr_att = np.zeros((2*data_size*ns,1))
xtr_att_truelabels = np.zeros((2*data_size*ns,))
model_target = Sequential()
model_target.add(Dense(nh, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
model_target.add(Dense(1, activation='sigmoid', name = 'output'))
opt = Adam(lr=lrate, decay=decay) 
model_target.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
print(model_target.summary())
hist_target = model_target.fit(xtr_target, ytr_target,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_target, yts_target), shuffle=True, verbose=0)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 8)                 128       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
print('\n\nFor target model with training datasize = %d'%data_size)
print('Training accuracy = ', hist_target.history['accuracy'][-1])
print('Validation accuracy = ', hist_target.history['val_accuracy'][-1])
model_target_name = 'Uber_Census_target_'+str(data_size)+'.h5'
model_target.save(model_target_name)
ytemp_tr_target = model_target.predict(xtr_target)
ytemp_ts_target = model_target.predict(xts_target)
xts_att = np.vstack((ytemp_tr_target,ytemp_ts_target))
yts_att = np.zeros((2*data_size,1))
yts_att[data_size:2*data_size] = 1  
xts_att_truelabels = np.vstack((ytr_target,yts_target))
xts_att_dict = {'xts_att':xts_att,'yts_att':yts_att,'xts_att_truelabels':xts_att_truelabels}
fname = './att_test_data_'+str(data_size)
np.save(fname,xts_att_dict)
datafile = './Uber_Census_target_'+str(data_size)
np.save(datafile,target_rep)



For target model with training datasize = 2250
Training accuracy =  0.7760000228881836
Validation accuracy =  0.7604444622993469


In [ ]:
for i in np.arange(ns):
    np.random.shuffle(sh1)
    shadow_rep[i,:] = sh1
    xtr_shadow = x[sh1[:data_size]]
    ytr_shadow = y[sh1[:data_size]]
    xts_shadow = x[sh1[data_size:2*data_size]]
    yts_shadow = y[sh1[data_size:2*data_size]]
    model_shadow = Sequential()
    model_shadow.add(Dense(nh, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
    model_shadow.add(Dense(1, activation='sigmoid', name = 'output'))
    opt = Adam(learning_rate=lrate, decay=decay) 
    model_shadow.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    if i == 0:
        print("Shadow Model Summary")
        print(model_shadow.summary())
    hist_shadow = model_shadow.fit(xtr_shadow, ytr_shadow,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_shadow, yts_shadow), shuffle=True, verbose=0)
    print("Shadow model no: %d"%i)
    print('\n\nFor shadow model with training datasize = %d'%data_size)
    print('Training accuracy = ', hist_shadow.history['accuracy'][-1])
    print('Validation accuracy = ', hist_shadow.history['val_accuracy'][-1])
    ytemp11 = model_shadow.predict(xtr_shadow)
    ytemp22 = model_shadow.predict(xts_shadow)
    model_shadow_name = 'Uber_Census_shadow_'+str(data_size)+'_'+str(i)+'.h5'
    print(model_shadow_name)
    model_shadow.save(model_shadow_name)    
    xtr_att[i*2*data_size:(i+1)*2*data_size] = np.vstack((ytemp11,ytemp22))
    ytr_att[((i*2)+1)*data_size:(i+1)*2*data_size] = 1
    xtr_att_truelabels[i*2*data_size:(i+1)*2*data_size] = np.hstack((ytr_shadow,yts_shadow))
datafile = './Uber_Census_shadow_'+str(data_size)
np.save(datafile,shadow_rep)
xtr_att_dict = {'xtr_att':xtr_att,'ytr_att':ytr_att,'xtr_att_truelabels':xtr_att_truelabels}
fname = './att_train_data_'+str(data_size)
np.save(fname,xtr_att_dict)

Shadow Model Summary
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 8)                 128       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________
None
Shadow model no: 0


For shadow model with training datasize = 2250
Training accuracy =  0.5053333044052124
Validation accuracy =  0.4897777736186981
Uber_Census_shadow_2250_0.h5
Shadow model no: 1


For shadow model with training datasize = 2250
Training accuracy =  0.5093333125114441
Validation accuracy =  0.4977777898311615
Uber_Census_shadow_2250_1.h5
Shadow model no: 2


For shadow model with training datasize = 2250


In [ ]:
print(xtr_att, xtr_att.shape)
print(ytr_att, ytr_att.shape)

print('****')
print(xts_att, xts_att.shape)
print(yts_att, yts_att.shape)

In [ ]:
model_attack = Sequential()
model_attack.add(Dense(nh, input_shape = (xtr_att.shape[1],), activation='sigmoid', name = 'hidden'))
model_attack.add(Dense(1, activation='sigmoid', name = 'output'))
opt = Adam(learning_rate=lrate, decay=decay) 
model_attack.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
print("Attack Model Summary")
print(model_attack.summary())
hist_attack = model_attack.fit(xtr_att, ytr_att,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data=(xts_att, yts_att), shuffle=True, verbose=0)
print('\n\nFor attack model with training datasize = %d'%xtr_att.shape[0])
print('Training accuracy = ', hist_shadow.history['accuracy'][-1])
print('Validation accuracy = ', hist_shadow.history['val_accuracy'][-1])